# AU demo using shapes_6dof dataset on PYNQ-Z2

This jupyter notebook runs the hardware demo of REMOT on shape_6dof dataset using 3 implementation: 
- FIFO-ONLY
- HASH-AMAP 
- Full-AMAP

Run through the notebook, the results of bbox, idx and video will be stored on the ./result folder

In [1]:
from sklearn.cluster import DBSCAN, AgglomerativeClustering
from scipy import io
from scipy.spatial.distance import squareform, directed_hausdorff
from itertools import combinations
import numpy as np
from pynq import Overlay 
from pynq import allocate
from pynq import Xlnk
import numpy as np
import time
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import math
import cv2
from tqdm import tqdm
from matplotlib import pyplot as plt
import argparse
import os
import csv
import yaml
from au_functions import *
from au_hardware_hash import Au_hash
from au_hardware_full import Au_full
from au_hardware_fifo_only import Au_fifo



# AU controller 

In [5]:

class Controller():
    def __init__(self, args):
        self.input = args.input
        self.bitfile = args.bitfile
        self.Init(self.input)
        self.tkBoxes = []
        self.tkIDs = []

        self.auFifo = args.auFifo
        self.auNum = args.auNum
        self.dAdd = args.dAdd
        self.folder = args.outfolder
        self.name = args.name

        self.tDel = args.tDel * self.tFrame
        self.areaDel = args.areaDel
        self.tLive = args.tLive * self.tFrame
        self.areaLive = args.areaLive
        self.numLive = args.numLive

        self.t = self.tFrame

        self.split = args.split
        self.epsDiv = args.epsDiv
        self.minptsDiv = 1

        self.merge = args.merge
        self.iomMer = args.iomMer
        self.dsMer = args.dsMer
        self.minptsMer = 1
        self.epsMer = 1

        if "inbound" in self.input:
            self.bdspawn1 = 40
            self.bdspawn2 = 40
        else:
            self.bdspawn1 = -1
            self.bdspawn2 = -1

        if "outbound" in self.input:
            self.bdkill = 20
        else:
            self.bdkill = -1

        self.globalID = -1
        if 'hash' in args.bitfile:
            self.AUs = Au_hash(Height=self.ly, Width=self.lx, bitfile=args.bitfile, au_number=args.auNum, fifo_depth=args.auFifo, dAdd=self.dAdd)
        elif 'full' in args.bitfile:
            self.AUs = Au_full(Height=260, Width=342, bitfile=args.bitfile, au_number=args.auNum, fifo_depth=args.auFifo, dAdd=self.dAdd)
        elif 'fifo' in args.bitfile:
            self.AUs = Au_fifo(Height=self.ly, Width=self.lx, bitfile=args.bitfile, au_number=args.auNum, fifo_depth=args.auFifo)
        else:
            raise ValueError("Not matching bitfile overlay")

        self.frame_count = 0
        x = self.events[:, 0]
        y = self.events[:, 1]
        p = self.events[:, 2]
        self.t = self.events[:, 3]
        self.total_time = self.t[-1]
        self.events = np.vstack([x, y, self.t, p]).T
            
            
    def Init(self, file):
        if "bound" in self.input:
            self.tFrame = 40000
        else:
            self.tFrame = 44065

        self.cmap = io.loadmat('cmap.mat')['cmap'] * 255
        self.events = io.loadmat(file)['events']
        self.lx, self.ly = self.events[:, :2].max(0) + 1
        self.pFrame = -1
        self.nFrame = self.events[:, 4][-1] + 1
        self.frame0 = np.zeros((self.ly, self.lx, 3), 'uint8')
        self.iFrame = self.frame0.copy()
        self.frames = np.tile(self.frame0, (self.nFrame, 1, 1, 1))

    def update_box(self, ts):
        live_au_list = np.where(self.AUs.status_reg == 0)[0]
        for i in live_au_list:
            auEvents = self.AUs.au_event_fifo[i]
            idxFade = np.argwhere(auEvents[:, 2] < ts - self.tFrame).flatten()

            idxFade = np.argwhere(auEvents[:, 2] < auEvents[-1, 2] - self.tFrame).flatten()
            if idxFade.size != 0:
                auEvents = np.delete(auEvents, idxFade, axis=0)
            self.AUs.au_event_fifo[i] = auEvents
            self.AUs.auBox[i] = bbox(auEvents[:, 0], auEvents[:, 1])
                
        
    def Split(self):
        idxDel = []
        if (np.sum(self.AUs.status_reg) == 0): 
            return 
        
        for j in np.where(self.AUs.status_reg != 1)[0]:
            auEvents = self.AUs.au_event_fifo[j] 
            if self.split == 'DBSCAN':
                idxGroup = DBSCAN(eps=self.epsDiv, min_samples=self.minptsDiv).fit_predict(
                    auEvents[:, :2])
                idxGroup[idxGroup < 0] = 0
            else:
                if au.auEvents.shape[0] <= 1:
                    continue
                clustering = AgglomerativeClustering(
                    linkage='average', affinity='euclidean').fit(auEvents[:, :2])
                idxGroup = clustering.labels_
                idxGroup[idxGroup < 0] = 0
                if max(directed_hausdorff(auEvents[idxGroup == 0, :2], auEvents[idxGroup == 1, :2])[0],
                       directed_hausdorff(auEvents[idxGroup == 1, :2], auEvents[idxGroup == 0, :2])[0]) < self.dsMer:
                    continue

            if max(idxGroup) <= 0: 
                continue
            else:
                idxDel.append(j)

            idxTk = np.argmax([sum(idxGroup == idx)
                              for idx in np.unique(idxGroup)])

            for k in range(max(idxGroup) + 1):
                next_empty = np.where(self.AUs.status_reg == 1)[0]
                if next_empty.shape[0] ==0:
                    return
                else:
                    next_empty = next_empty[0]
                    
                    idxEvents = np.argwhere(idxGroup == k).flatten()
                    event_collect = auEvents[idxEvents]
                    self.AUs.auBox[next_empty] = bbox(event_collect[:, 0], event_collect[:, 1]) 
                    
                    self.AUs.write_au(event=event_collect, number=next_empty)
                    if k == idxTk:
                        self.AUs.auNumber[next_empty] = self.AUs.auNumber[j] 
                    else:
                        self.AUs.auNumber[next_empty] = [0, min(event_collect[:, 2])]

        if len(idxDel) > 0:
            for idx in (idxDel):
                self.AUs.kill_au(idx)
                
    def Merge(self):
        live_au_list = np.where(self.AUs.status_reg == 0)[0]
        if live_au_list.shape[0] <= 1: ## if state reg only has one 0
            return
        
        idxnk = list(combinations(live_au_list, 2)) 
        idxGroup = clusterAu(np.array(self.AUs.auBox)[live_au_list], self.iomMer)

        for j in range(max(idxGroup) + 1):
            idxAU = np.argwhere(idxGroup == j).flatten()
            idxAU = live_au_list[idxAU]
            idxDel = idxAU
            if idxAU.size < 2:
                continue
            
            write_au_idx = np.min(idxAU)
            print("merging{} to {}".format(idxAU, write_au_idx) )
            events = np.concatenate(
                [self.AUs.au_event_fifo[idx] for idx in idxAU], axis=0) 
            events = np.unique(events, axis=0)
            events = events[np.argsort(events[:, 2])]
        
            if any([self.AUs.auNumber[idx][0] > 0 for idx in idxAU]):
                idxAU = idxAU[[self.AUs.auNumber[idx][0] > 0 for idx in idxAU]]
            idxNum = idxAU[np.argmin([self.AUs.auNumber[idx][0] for idx in idxAU])]
            
            self.AUs.write_au(event=events, number=write_au_idx)
            self.AUs.auBox[write_au_idx] = bbox(events[:, 0], events[:, 1])
            self.AUs.auNumber[write_au_idx] = self.AUs.auNumber[idxNum]
            
            for idx in idxDel:
                if idx == write_au_idx:
                    continue
                self.AUs.kill_au(idx)

    def Kill(self, ts):
        live_au_list = np.where(self.AUs.status_reg==0)[0]
        idxDel = []
        
        for idx in live_au_list:
            flag1 = ts - np.max(self.AUs.au_event_fifo[idx][:, 2]) > self.tDel
            flag2 = bbArea(self.AUs.auBox[idx]) < self.areaDel
            flag3 = (self.AUs.auBox[idx][1] + self.AUs.auBox[idx][3]) / 2 < self.bdkill
            if flag1 or flag2 or flag3:
                idxDel.append(idx)
                                 
                              
        if len(idxDel) > 0:
            print("killing", idxDel)
            for idx in idxDel:
                self.AUs.kill_au(idx)  

    def UpdateID(self, ts):
        if 'fifo' in self.bitfile:
            self.AUs.write_all_au()
            
        live_au_list = np.where(self.AUs.status_reg==0)[0]
        for idx in live_au_list:
            if not self.AUs.auNumber[idx][0] and \
            ts - self.AUs.auNumber[idx][1] > self.tLive and \
            bbArea(self.AUs.auBox[idx]) > self.areaLive and \
            self.AUs.au_event_fifo[idx].shape[0] > self.numLive and \
            self.AUs.auBox[idx][2] / 2 > self.bdspawn1 and \
            self.AUs.auBox[idx][3] / 2 > self.bdspawn2:
                self.globalID += 1
                self.AUs.auNumber[idx][0] = self.globalID

    def Animation(self, events, ts):
        self.iFrame[:] = 0
        y = events[:, 1]
        x = events[:, 0]
        self.iFrame[y, x] = [255, 255, 255]
        
        idxVis = []
        boxes = []
        IDs = []

        live_au_list = np.where(self.AUs.status_reg == 0)[0]
        cmap_idx = np.array([self.AUs.auNumber[i][0] % 7 for i in live_au_list], 'int32')
        auColors = np.zeros([self.auNum, 3])
        auColors[live_au_list] = self.cmap[cmap_idx]

        for j in live_au_list:
            if self.AUs.auNumber[j][0] > 0:
                idxEvt = self.AUs.au_event_fifo[j][:, 2] >= ts - self.tFrame
                if any(idxEvt):
                    idxVis.append(j)
                    one_frame_events = self.AUs.au_event_fifo[j][idxEvt, :]
                    boxes.append(
                        bbox(one_frame_events[:, 0], one_frame_events[:, 1]))
                    IDs.append(self.AUs.auNumber[j][0])

        self.tkBoxes.append(boxes)
        self.tkIDs.append(IDs)

        if len(idxVis) > 0:
            for j, k in enumerate(idxVis):
                self.iFrame = cv2.rectangle(
                    self.iFrame, (boxes[j][0], boxes[j][1]), (boxes[j][2], boxes[j][3]), auColors[k].tolist(), 1)

        if len(idxVis) > 0:
            for j, k in enumerate(idxVis):
                self.iFrame = cv2.putText(self.iFrame, '{}'.format(
                    IDs[j]), (boxes[j][0], boxes[j][1]), cv2.FONT_HERSHEY_PLAIN, 1., auColors[k].tolist(), 1)


        self.frames[self.frame_count] = self.iFrame


    def SaveResults(self):
        if not os.path.exists(self.folder):
            os.mkdir(self.folder)

        tkbox_dir = os.path.join(self.folder, self.name + 'tkBoxes.mat')
        tkid_dir = os.path.join(self.folder, self.name + 'tkIDs.mat')
        gt_dir = self.input.replace('.mat', 'GT.mat')
        video_dir = os.path.join(self.folder, self.name + 'output.avi')

        for boxes in self.tkBoxes:
            if len(boxes) == 0:
                continue
            for box in boxes:
                box[2] = box[2] - box[0]
                box[3] = box[3] - box[1]

        self.tkBoxes = np.array(self.tkBoxes, dtype='object')
        self.tkIDs = np.array(self.tkIDs, dtype='object')
        io.savemat(tkbox_dir, {'tkBoxes': self.tkBoxes})
        io.savemat(tkid_dir, {'tkIDs': self.tkIDs})

        fourcc = cv2.VideoWriter_fourcc(*'XVID')
        videoWriter = cv2.VideoWriter(
            video_dir, fourcc, 5, (self.lx, self.ly), True)
        for i in range(self.frame_count):
            videoWriter.write(self.frames[i])
        videoWriter.release()
        print("Saving results in ", tkbox_dir, tkid_dir, video_dir)

                
    def Process(self, save_result=True):
        
        for i in range(self.total_time // (1 * self.tFrame)):
            print("Processing {}---------------------------------------".format(self.frame_count))
            idxs = np.where((self.t > self.frame_count * self.tFrame) & (self.t <= (self.frame_count + 1) * 1 * self.tFrame))[0]
            events = self.events[idxs]
            ts = events[-1, 2]
            self.AUs.stream_in_events(events)
            self.AUs.dump_all_au()
            self.update_box(ts)
            
            self.Split()
            self.Merge()
            self.Kill(ts)
            self.UpdateID(ts)    
            if save_result:
                self.Animation(events, ts)
            self.frame_count += 1

        
        if save_result:
            self.SaveResults()


    
            
 

# FIFO-only

In [6]:
config_dir = "./config/shape_6dof_fifo.yml"

with open(config_dir, "r") as file:
    config = yaml.safe_load(file)

config = ARGS(config)
controller = Controller(config)
controller.Process()

Processing 0---------------------------------------
processing 226 event using:0.000469207763671875
killing [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
Processing 1---------------------------------------
processing 997 event using:0.0006568431854248047
merging[0 5] to 0
merging[ 1  2 11] to 1
merging[3 7] to 3
killing [1, 4, 9]
Processing 2---------------------------------------
processing 1084 event using:0.0008165836334228516
merging[0 4 8] to 0
merging[2 6] to 2
merging[3 5] to 3
killing [7, 11]
Processing 3---------------------------------------
processing 1352 event using:0.0006814002990722656
merging[1 5] to 1
merging[ 4  7 10] to 4
killing [11]
Processing 4---------------------------------------
processing 674 event using:0.0008790493011474609
merging[ 4 11] to 4
merging[6 9] to 6
killing [7, 8, 10]
Processing 5---------------------------------------
processing 196 event using:0.0006272792816162109
merging[0 9] to 0
merging[1 8] to 1
merging[ 4 10] to 4
killing [5, 7, 11]
Processing 

merging[0 3] to 0
merging[ 9 10] to 9
killing [9, 11]
Processing 54---------------------------------------
processing 4630 event using:0.001800537109375
merging[ 2  3  9 10 11] to 2
killing [6]
Processing 55---------------------------------------
processing 4399 event using:0.0017199516296386719
merging[ 2  3  9 10 11] to 2
Processing 56---------------------------------------
processing 4127 event using:0.001628875732421875
merging[ 2  3  9 10] to 2
merging[ 4 11] to 4
merging[6 8] to 6
Processing 57---------------------------------------
processing 3340 event using:0.001356363296508789
merging[ 1 11] to 1
merging[ 2  3  8  9 10] to 2
Processing 58---------------------------------------
processing 1968 event using:0.0009212493896484375
merging[2 3 8] to 2
merging[10 11] to 10
killing [9, 10]
Processing 59---------------------------------------
processing 991 event using:0.0006229877471923828
merging[0 8] to 0
merging[ 2  3  9 10] to 2
merging[ 7 11] to 7
Processing 60------------------

merging[ 7 11] to 7
killing [6, 9, 10]
Processing 108---------------------------------------
processing 6480 event using:0.0024111270904541016
merging[ 3 10] to 3
merging[4 8] to 4
killing [0, 9]
Processing 109---------------------------------------
processing 5816 event using:0.002184152603149414
merging[ 0  9 11] to 0
merging[5 6] to 5
merging[7 8] to 7
killing [2, 10]
Processing 110---------------------------------------
processing 3492 event using:0.0014109611511230469
merging[ 0  7 10] to 0
merging[ 5 11] to 5
killing [2, 6, 8, 9]
Processing 111---------------------------------------
processing 5773 event using:0.002195119857788086
merging[ 0 11] to 0
merging[1 8] to 1
merging[ 5  7  9 10] to 5
killing [2, 6]
Processing 112---------------------------------------
processing 5675 event using:0.0021364688873291016
merging[0 6] to 0
merging[2 4] to 2
killing [7, 8, 9, 10, 11]
Processing 113---------------------------------------
processing 4268 event using:0.0016744136810302734
mergin

merging[0 7] to 0
merging[ 1 11] to 1
merging[4 8] to 4
killing [5]
Processing 164---------------------------------------
processing 13118 event using:0.0045545101165771484
merging[0 5] to 0
merging[ 4 11] to 4
merging[ 7  8 10] to 7
Processing 165---------------------------------------
processing 9528 event using:0.003385782241821289
merging[0 8] to 0
merging[1 5] to 1
merging[ 3 10] to 3
killing [11]
Processing 166---------------------------------------
processing 7868 event using:0.0028429031372070312
merging[ 1 10] to 1
merging[2 5] to 2
merging[4 8] to 4
merging[ 6 11] to 6
Processing 167---------------------------------------
processing 7449 event using:0.002698183059692383
killing [5, 8, 10, 11]
Processing 168---------------------------------------
processing 2840 event using:0.0011925697326660156
merging[ 1 11] to 1
merging[6 8] to 6
killing [5, 9, 10]
Processing 169---------------------------------------
processing 4080 event using:0.0016124248504638672
merging[ 3  5  9 11] to

merging[7 9] to 7
Processing 221---------------------------------------
processing 4306 event using:0.0016868114471435547
merging[0 9] to 0
merging[ 7 10] to 7
killing [11]
Processing 222---------------------------------------
processing 4414 event using:0.0017247200012207031
merging[ 6  9 10] to 6
Processing 223---------------------------------------
processing 4121 event using:0.001611948013305664
merging[ 0 10] to 0
merging[ 6 11] to 6
killing [9]
Processing 224---------------------------------------
processing 4955 event using:0.001920938491821289
merging[ 9 11] to 9
killing [9, 10]
Processing 225---------------------------------------
processing 3411 event using:0.0013747215270996094
merging[ 1 10 11] to 1
merging[3 9] to 3
Processing 226---------------------------------------
processing 3417 event using:0.0014004707336425781
merging[ 1 10 11] to 1
merging[5 9] to 5
Processing 227---------------------------------------
processing 3816 event using:0.0015118122100830078
merging[ 1  

# FULL AMAP

In [7]:
config_dir = "./config/shape_6dof_full.yml"

with open(config_dir, "r") as file:
    config = yaml.safe_load(file)

config = ARGS(config)

controller = Controller(config)
controller.Process()

Processing 0---------------------------------------
processing 226 event using:0.0005085468292236328
killing [1]
Processing 1---------------------------------------
processing 997 event using:0.0013899803161621094
Processing 2---------------------------------------
processing 1084 event using:0.0014941692352294922
Processing 3---------------------------------------
processing 1352 event using:0.0018124580383300781
Processing 4---------------------------------------
processing 674 event using:0.0010406970977783203
Processing 5---------------------------------------
processing 196 event using:0.0006499290466308594
Processing 6---------------------------------------
processing 865 event using:0.0012888908386230469
Processing 7---------------------------------------
processing 806 event using:0.0012259483337402344
Processing 8---------------------------------------
processing 1624 event using:0.0021381378173828125
Processing 9---------------------------------------
processing 1906 event us

processing 6857 event using:0.008185863494873047
Processing 82---------------------------------------
processing 6454 event using:0.007722139358520508
Processing 83---------------------------------------
processing 6620 event using:0.008037328720092773
Processing 84---------------------------------------
processing 5944 event using:0.007137775421142578
Processing 85---------------------------------------
processing 4431 event using:0.005388021469116211
Processing 86---------------------------------------
processing 4253 event using:0.005198240280151367
Processing 87---------------------------------------
processing 4838 event using:0.00586247444152832
Processing 88---------------------------------------
processing 5068 event using:0.006131887435913086
Processing 89---------------------------------------
processing 5029 event using:0.0060977935791015625
Processing 90---------------------------------------
processing 4638 event using:0.0056493282318115234
Processing 91-------------------

processing 8695 event using:0.01032876968383789
Processing 162---------------------------------------
processing 10405 event using:0.012294292449951172
Processing 163---------------------------------------
processing 9944 event using:0.011757612228393555
Processing 164---------------------------------------
processing 13118 event using:0.015424489974975586
Processing 165---------------------------------------
processing 9528 event using:0.011281013488769531
Processing 166---------------------------------------
processing 7868 event using:0.009367227554321289
Processing 167---------------------------------------
processing 7449 event using:0.00888967514038086
Processing 168---------------------------------------
processing 2840 event using:0.003538846969604492
Processing 169---------------------------------------
processing 4080 event using:0.005013227462768555
Processing 170---------------------------------------
processing 2772 event using:0.003485441207885742
Processing 171----------

# HASH-AMAP

In [8]:
config_dir = "./config/shape_6dof_hash.yml"

with open(config_dir, "r") as file:
    config = yaml.safe_load(file)

config = ARGS(config)
    
controller = Controller(config)
controller.Process()

Processing 0---------------------------------------
processing 226 event using:0.0005323886871337891
merging[0 7] to 0
killing [1, 2, 3, 4, 6, 8, 9]
Processing 1---------------------------------------
processing 997 event using:0.0015463829040527344
merging[1 4 7] to 1
merging[2 8] to 2
merging[3 6] to 3
Processing 2---------------------------------------
processing 1084 event using:0.0016853809356689453
merging[2 4 8] to 2
Processing 3---------------------------------------
processing 1352 event using:0.0020513534545898438
merging[4 6] to 4
merging[7 8] to 7
Processing 4---------------------------------------
processing 674 event using:0.0012178421020507812
merging[2 8] to 2
merging[3 5] to 3
merging[6 9] to 6
Processing 5---------------------------------------
processing 196 event using:0.00058746337890625
merging[3 9] to 3
killing [5, 8]
Processing 6---------------------------------------
processing 865 event using:0.0014045238494873047
merging[1 5] to 1
merging[2 8] to 2
killing [9

merging[2 8] to 2
killing [9]
Processing 64---------------------------------------
processing 2993 event using:0.00429224967956543
merging[2 8 9] to 2
merging[4 7] to 4
Processing 65---------------------------------------
processing 2637 event using:0.0038275718688964844
merging[2 8 9] to 2
merging[3 7] to 3
Processing 66---------------------------------------
processing 3062 event using:0.004380226135253906
merging[1 7] to 1
merging[2 8 9] to 2
Processing 67---------------------------------------
processing 4290 event using:0.006024837493896484
merging[2 8 9] to 2
killing [7]
Processing 68---------------------------------------
processing 4792 event using:0.006677150726318359
merging[2 9] to 2
merging[3 8] to 3
killing [7]
Processing 69---------------------------------------
processing 4391 event using:0.006131887435913086
merging[2 8] to 2
killing [9]
Processing 70---------------------------------------
processing 5247 event using:0.007275819778442383
merging[2 8] to 2
killing [9]
Pr

processing 8399 event using:0.011471986770629883
killing [8, 9]
Processing 127---------------------------------------
processing 6255 event using:0.008723735809326172
merging[6 9] to 6
killing [5, 8]
Processing 128---------------------------------------
processing 8983 event using:0.01232600212097168
merging[3 5] to 3
merging[6 9] to 6
Processing 129---------------------------------------
processing 7510 event using:0.010296821594238281
merging[5 9] to 5
Processing 130---------------------------------------
processing 7501 event using:0.010274648666381836
killing [9]
Processing 131---------------------------------------
processing 10651 event using:0.014419078826904297
killing [9]
Processing 132---------------------------------------
processing 8148 event using:0.011125802993774414
Processing 133---------------------------------------
processing 9348 event using:0.012740135192871094
Processing 134---------------------------------------
processing 11697 event using:0.015867233276367188


killing [7]
Processing 196---------------------------------------
processing 11849 event using:0.01606607437133789
Processing 197---------------------------------------
processing 6825 event using:0.00937652587890625
Processing 198---------------------------------------
processing 8781 event using:0.01198124885559082
Processing 199---------------------------------------
processing 12715 event using:0.017227888107299805
Processing 200---------------------------------------
processing 7875 event using:0.010774612426757812
Processing 201---------------------------------------
processing 8933 event using:0.01218414306640625
Processing 202---------------------------------------
processing 7868 event using:0.01074075698852539
Processing 203---------------------------------------
processing 2371 event using:0.003424406051635742
Processing 204---------------------------------------
processing 5725 event using:0.007884740829467773
Processing 205---------------------------------------
processing